stepwise variable selection using p-value
-----------------------------------------------------------
**0.9522**

In [4]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

import numpy as np
import pandas as pd

In [5]:
def read_data():
    x_train, y_train, x_test = [], [], []
    with open('a3-train-features.txt', 'rt') as f:
        for line in f:
            x_train.append([float(s) for s in line.strip().split('\t')])
    with open('a3-train-outcome.txt.', 'rt') as f:
        for line in f:
            y_train.append(float(line.strip()))
    with open('a3-test-features.txt', 'rt') as f:
        for line in f:
            x_test.append([float(s) for s in line.strip().split('\t')])
    return np.array(x_train), np.array(y_train), np.array(x_test)

In [6]:
def scale(data):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    
    return data_scaled

In [7]:
def stepwise_variable_selection(x_train, y_train, th_in = 0.01, th_out = 0.05, 
                               ):
    x_train = pd.DataFrame(x_train, 
                           columns = [f'variable_{i}' for i in range(x_train.shape[1])])
    variables = x_train.columns
    
    sl_vars = []
    sv_per_step = []
    adjusted_r_squared = []
    steps = []
    step = 0
    
    
    while len(variables) > 0:
        remainder = list(set(variables) - set(sl_vars))
        pval = pd.Series(index = remainder)
    
        for col in remainder:
            X = x_train.loc[:, sl_vars + [col]]
            X = sm.add_constant(X)
            model = sm.OLS(y_train, X).fit(disp = 0)
            pval.loc[col] = model.pvalues[-1]
        
        if pval.min() < th_in:
            sl_vars.append(pval.idxmin())
    
            while len(sl_vars) > 0:
                X = x_train.loc[:, sl_vars]
                X = sm.add_constant(X)
                sl_pval = pd.Series(sm.OLS(y_train, X).fit(disp = 0).pvalues[1:], 
                                    index = sl_vars)
                
                if sl_pval.max() >= th_out:
                    sl_vars.remove(sl_pval.idxmax())
                else:
                    break

            
            step += 1
            steps.append(step)
            adj_r_squared = sm.OLS(y_train, sm.add_constant(x_train.loc[:, sl_vars])).fit(disp = 0).rsquared_adj
            adjusted_r_squared.append(adj_r_squared)
            sv_per_step.append(sl_vars.copy())
        else:
            break 
                
    return sl_vars

In [46]:
if __name__ == '__main__':
    
    x_train, y_train, x_test = read_data()
    x_train, x_test = pd.DataFrame(x_train, columns = [f'variable_{i}' for i in range(x_train.shape[1])]), pd.DataFrame(x_test, columns = [f'variable_{i}' for i in range(x_test.shape[1])]) 

    index = stepwise_variable_selection(x_train, y_train)
    x_train, x_test = np.array(x_train.loc[:, index]), np.array(x_test.loc[:, index])
    
    alphas = np.logspace(-4, 4, 100)
    lasso_cv = LassoCV(alphas = alphas, cv = 5, random_state = 42, max_iter = 3000)
    lasso_cv.fit(x_train, y_train)
    pred = lasso_cv.predict(x_test)
    
    with open('predictions.txt', 'wt') as fout:
        for p in pred:
            print(p, file = fout)     

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

...

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/229459030.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

-------------------------------------------------------
pvalue로 stepwise variable selection 하면 5개 column

In [48]:
sl_vars

['variable_69', 'variable_70', 'variable_41', 'variable_61', 'variable_76']

------------------------------------

stepwise variable selection function
scale 없이

In [167]:
X_train

array([[-1.57585348e+01, -3.21359192e+00, -3.06583987e+00, ...,
        -3.57856701e+01, -1.69745245e-02, -7.70083355e+00],
       [-1.24725286e+01, -4.61894573e+00, -1.40172986e+00, ...,
        -6.02781671e+00, -4.73591136e-01, -4.74642168e+00],
       [-3.45654912e+00, -6.21960669e+00, -2.00575123e+00, ...,
        -1.92240135e+00, -4.13022761e-01, -8.16566228e+00],
       ...,
       [-1.06841403e+01, -5.52509019e+00, -8.47952468e-01, ...,
        -2.88015245e+01, -8.23250328e-01, -2.88360423e+00],
       [-1.70460987e+01, -6.49967789e-01, -5.29721112e-01, ...,
        -2.24313222e+01, -8.12888553e-01, -7.27778688e-01],
       [-1.05085486e+01, -4.59923680e+00, -2.23406364e+00, ...,
        -4.97052003e+00, -2.78569836e-01, -1.77066604e+01]])

In [173]:
index

['variable_69', 'variable_41', 'variable_70', 'variable_37']

In [179]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train)
index = stepwise_variable_selection(X_train, Y_train)

X_train, X_val = pd.DataFrame(X_train, columns = [f'variable_{i}' for i in range(X_train.shape[1])]), pd.DataFrame(X_val, columns = [f'variable_{i}' for i in range(X_val.shape[1])]) 

X_train, X_val = np.array(X_train.loc[:, index]), np.array(X_val.loc[:, index])

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

...

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

In [180]:
alphas = np.logspace(-4, 4, 100)
lasso_cv = LassoCV(alphas = alphas, cv = 10, random_state = 42, max_iter = 3000)
lasso_cv.fit(X_train, Y_train)
pred = lasso_cv.score(X_val, Y_val)

In [181]:
pred

0.9724427862960011

--------------------------------------------------------
stepwise variable selection scale 하고

In [187]:
def scalen_stepwise_variable_selection(x_train, y_train, th_in = 0.01, th_out = 0.05, 
                               ):
    x_train = pd.DataFrame(scale(x_train), 
                           columns = [f'variable_{i}' for i in range(x_train.shape[1])])
    variables = x_train.columns
    
    sl_vars = []
    sv_per_step = []
    adjusted_r_squared = []
    steps = []
    step = 0
    
    
    while len(variables) > 0:
        remainder = list(set(variables) - set(sl_vars))
        pval = pd.Series(index = remainder)
    
        for col in remainder:
            X = x_train.loc[:, sl_vars + [col]]
            X = sm.add_constant(X)
            model = sm.OLS(y_train, X).fit(disp = 0)
            pval.loc[col] = model.pvalues[-1]
        
        if pval.min() < th_in:
            sl_vars.append(pval.idxmin())
    
            while len(sl_vars) > 0:
                X = x_train.loc[:, sl_vars]
                X = sm.add_constant(X)
                sl_pval = pd.Series(sm.OLS(y_train, X).fit(disp = 0).pvalues[1:], 
                                    index = sl_vars)
                
                if sl_pval.max() >= th_out:
                    sl_vars.remove(sl_pval.idxmax())
                else:
                    break

            
            step += 1
            steps.append(step)
            adj_r_squared = sm.OLS(y_train, sm.add_constant(x_train.loc[:, sl_vars])).fit(disp = 0).rsquared_adj
            adjusted_r_squared.append(adj_r_squared)
            sv_per_step.append(sl_vars.copy())
        else:
            break 
                
    return sl_vars

In [188]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train)
index = stepwise_variable_selection(X_train, Y_train)

X_train, X_val = pd.DataFrame(X_train, columns = [f'variable_{i}' for i in range(X_train.shape[1])]), pd.DataFrame(X_val, columns = [f'variable_{i}' for i in range(X_val.shape[1])]) 

X_train, X_val = np.array(X_train.loc[:, index]), np.array(X_val.loc[:, index])

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

...

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = model.pvalues[-1]
C:\Users\AARONC~1\AppData\Local\Temp/ipykernel_27128/927169215.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pval.loc[col] = mode

In [189]:
alphas = np.logspace(-4, 4, 100)
lasso_cv = LassoCV(alphas = alphas, cv = 10, random_state = 42, max_iter = 3000)
lasso_cv.fit(X_train, Y_train)
pred = lasso_cv.score(X_val, Y_val)

In [190]:
pred

0.9741705559334479